# Klasifikasi Ulasan Online Shop
Berikut ini proses untuk mengelompokkan Ulasan (Feed Back) di Online Shop berdasarkan Label tingkat kepuasan konsumen dengan
menggunakan Algoritma Knn.

### Label Tingkat Kepuasan
    => 0 = Tidak  Puas
    => 1 = Kurang Puas
    => 2 = Cukup  Puas
    => 3 = Sangat Puas

### Ref:
   - https://github.com/andrepamungkas/klasifikasi-sms-knn
   - https://bukalapak.com
   - https://tokopedia.com
   - https://shoope.co.id

## Inisialisasi library 

In [1]:
import string
import re
import json
from collections import Counter

# sklearn - machine learning library
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

# numpy - scientific computing library
import numpy as np

# pandas - python data analysis library
import pandas as pd

# sastrawi - stemming library (bahasa indonesia)
#pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# tqdm - progress bar library
from tqdm import tqdm

## Preprocessing
Membersihkan dokumen dengan cara:
1. Menghilangkan kata yang tidak penting
2. Menghilangkan tanda baca
3. Mengubah kata ke bentuk dasar
4. Menghilangkan angka

In [15]:
# import file yang berisi kata-kata tidak penting
data_stopword = json.load(open('stopwords-id.json','r'))

# menjadikan array stopword menjadi unordered collection (set())
# agar dapat dilakukan operasi matematis seperti union, intersection, symmetric difference
stopword = set(data_stopword)
punctuation = set(string.punctuation)

# method untuk cleaning dokumen
def clean(doc):
    # menghilangkan kata tidak penting
    stop_free = " ".join([i for i in doc.lower().split() if i not in stopword])
    # menghilangkan tanda baca
    punc_free = ''.join(ch for ch in stop_free if ch not in punctuation)
    # menjadikan ke kata dasar
    stemmer = StemmerFactory().create_stemmer()
    normalized = stemmer.stem(punc_free)
    # menghilangkan angka
    processed = re.sub(r"\d+","",normalized)
    # membuat satu dokumen menjadi array berisi tiap kata
    y = processed.split()
    return y

# method untuk cleaning dokumen berupa array
def clean_with_loop(arr):
    hasil = []
    progress = tqdm(arr)
    for item in progress:
        progress.set_description("Membersihkan dokumen")
        cleaned = clean(item)
        cleaned = ' '.join(cleaned)
        hasil.append(cleaned)
    return hasil

In [16]:
# import dataset olshop
olshop_csv = pd.read_csv('dataset_olshop3.csv')
print(olshop_csv.head())

# mengambil hanya kolom Teks olshop dan disimpan di variabel olshop (yang siap dibersihkan)
olshop = []
for index, row in olshop_csv.iterrows():
    olshop.append(row["Komentar"])
print("Jumlah Komentar: ", len(olshop))

# mengambil hanya kolom label dalam variabel y_train
y_train = []
for index, row in olshop_csv.iterrows():
    y_train.append(row["Label"])
print("Jumlah Label: ", len(y_train))

# membersihkan dokumen olshop
olshop_bersih = clean_with_loop(olshop)

                                            Komentar  Label
0         Lama pengiriman. Chat lama tidak si respon      0
1                   Kecepatan pengiriman tidak baik.      0
2  Barang Belum saya Terima gak tau tuh kondisi b...      0
3  Kualitas produk tidak baik. Kecepatan pengirim...      0
4  Pengiriman Super Duperr LAMAA!!! Pas ditanya a...      0
Jumlah Komentar:  300
Jumlah Label:  300


Membersihkan dokumen: 100%|██████████████████████████████████████████████████████████| 300/300 [02:44<00:00,  1.61it/s]


## Membentuk TF-IDF

In [17]:
# pembentukan vektor tf-idf untuk pembobotan kata
vectorizer = TfidfVectorizer(stop_words=data_stopword)
x_train = vectorizer.fit_transform(olshop_bersih)
print(x_train)

  (0, 435)	0.6340464614196564
  (0, 392)	0.223453997314607
  (0, 158)	0.4107604146663474
  (0, 833)	0.22758319973940824
  (0, 748)	0.49782062895428575
  (0, 673)	0.2823292159384112
  (1, 392)	0.5179369945820095
  (1, 833)	0.5275079430529434
  (1, 153)	0.6734067416201059
  (2, 65)	0.2901182540228691
  (2, 820)	0.13338051480298993
  (2, 251)	0.15332971717384053
  (2, 806)	0.17560149845903106
  (2, 866)	0.2450703738925918
  (2, 407)	0.18857856098172784
  (2, 387)	0.21682446743715986
  (2, 799)	0.21682446743715986
  (2, 798)	0.2450703738925918
  (2, 514)	0.15786218729401252
  (2, 620)	0.15123950367273206
  (2, 741)	0.10320904246101767
  (2, 284)	0.20030167136185453
  (2, 458)	0.415462633167192
  (2, 115)	0.21682446743715986
  (2, 770)	0.2450703738925918
  :	:
  (296, 478)	0.2601050751751309
  (296, 781)	0.29067696945664856
  (296, 21)	0.3063930841469272
  (296, 216)	0.3285437037097858
  (296, 76)	0.3285437037097858
  (296, 747)	0.3285437037097858
  (296, 762)	0.3285437037097858
  (297, 165

### Note!

Kalau error `IOPub data rate exceeded`
1. Buka **Anaconda Prompt**
2. Jalankan perintah `jupyter notebook --generate-config` untuk generate file config
3. Buka file config yang lokasinya ditampilkan setelah step 2 dieksekusi
4. Cari `c.NotebookApp.iopub_data_rate_limit`
5. Hapus `#` di depan dan ubah nilainya `10000000` (tambah 0 satu kali)

In [18]:
# pengelompokan dokumen dengan knn (k=5)
# penghitungan jarak dengan euclidean distance
# dokumentasi: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
modelknn = KNeighborsClassifier(n_neighbors=5, weights='distance')
modelknn.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='distance')

## Pengujian

In [20]:
kalimat_tes = ["Lama sampainya chat respon lama",
               "barang diterima dengan baik, tetapi kualitas nya tidak sesuai dengan perkiraan saya",
               "Kualitas produk standar. Harga produk standar.",
               "alhamdulillah barang berjalan dengan baik,, terima kasih"
              ]

# membersihkan dokumen pengujian
kalimat_tes_bersih = clean_with_loop(kalimat_tes)

# definisikan nama label
nama_label = ["Tidak","Kurang","Cukup" ,"Sangat"]

# loop untuk prediksi kelompok
for teks in kalimat_tes_bersih:
    arr_teks = []
    arr_teks.append(teks)
    vektor = vectorizer.transform(arr_teks)
    prediksi_label_knn = modelknn.predict(vektor)
    print(teks, ":\n" + "- Rating: " + nama_label[np.int(prediksi_label_knn)]+ " Puas\n")

Membersihkan dokumen: 100%|██████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  6.97it/s]


lama sampai chat respon lama :
- Rating: Tidak Puas

barang terima baik kualitas nya tidak sesuai kira :
- Rating: Kurang Puas

kualitas produk standar harga produk standar :
- Rating: Cukup Puas

alhamdulillah barang jalan baik terima kasih :
- Rating: Sangat Puas

